In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [59]:
import numpy as np
import pandas as pd
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [60]:
data = pd.read_csv('../input/digit-recognizer/train.csv')

In [61]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_test = data[0:1000].T
Y_test = data_test[0]
X_test = data_test[1:n]
X_test = X_test/255

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255

In [63]:
Y_train.shape

(41000,)

In [64]:
def init_params():
    W1 = np.random.uniform(-0.5, 0.5, (120, 784))
    b1 = np.random.uniform(-.5, 0.5, (120, 1))
    W2 = np.random.uniform(-0.5, 0.5, (40, 120))
    b2 = np.random.uniform(-0.5, 0.5, (40, 1))
    W3 = np.random.uniform(-0.5, 0.5, (10, 40))
    b3 = np.random.uniform(-0.5, 0.5, (10, 1))
    K = np.random.uniform(-0.5, 0.5, (3,))
    return W1, b1, W2, b2, W3, b3, K

In [65]:
def activation(k0, k1, z):
    return k0 + np.multiply(k1, z)

In [66]:
def softmax(Z):
    f_x = np.exp(Z) / np.sum(np.exp(Z))
    return f_x

In [67]:
def forward_propogation(W1, b1, W2, b2, W3, b3, k0, k1, X):
    Z1 = W1.dot(X) + b1
    A1 = activation(k0, k1, Z1)
    Z2 = W2.dot(A1) + b2
    A2 = activation(k0, k1, Z2)
    Z3 = W3.dot(A2) + b3
    A3 = softmax(Z3)
    return Z1, A1, Z2, A2, Z3, A3

In [68]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [69]:
def deri_act(k0, k1, z):
    return k1

In [70]:
def backward_propogation(Z1, A1, Z2, A2, A3, W3, W2, k0, k1, X, Y):
    m = Y.size
    
    one_hot_Y = one_hot(Y)
    dZ3 = A3 - one_hot_Y
    dW3 = 1/m * dZ3.dot(A2.T)
    db3 = 1/m * np.sum(dZ3)
    
    dA2 = W3.T.dot(dZ3)
    dZ2 = W3.T.dot(dZ3) * deri_act(k0, k1, Z2)
    dW2 = 1/m * dZ2.dot(A1.T)
    db2 = 1/m * np.sum(dZ2)
    dK2 = [np.sum(dA2)/ dA2.size, np.sum(dZ2.dot(dA2.T)) / (dZ2.dot(dA2.T)).size, np.sum((dZ2.T).dot(dA2.dot(dA2.T))) / ((dZ2.T).dot(dA2.dot(dA2.T))).size]
    
    dA1 = W2.T.dot(dZ2)
    dZ1 = W2.T.dot(dZ2) * deri_act(k0, k1, Z1)
    dW1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * np.sum(dZ1)
    dK1 = [np.sum(dA1)/ dA1.size, np.sum(dZ1.dot(dA1.T))/(dZ1.dot(dA1.T)).size, np.sum((dZ1.T).dot(dA1.dot(dA1.T)))/(dZ1.T).dot(dA1.dot(dA1.T)).size]
    
    dK = np.add(dK1, dK2)
    return dW1, db1, dW2, db2, dW3, db3, dK

In [71]:
def update_params(W1, b1, W2, b2, W3, b3, K, dW1, db1, dW2, db2, dW3, db3, dK, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    W3 = W3 - alpha * dW3
    b3 = b3 - alpha * db3
    K = K - alpha * dK
    return W1, b1, W2, b2, W3, b3, K

In [72]:
def get_predictions(A3):
    return np.argmax(A3, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

In [73]:
def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2, W3, b3, K = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2, Z3, A3 = forward_propogation(W1, b1, W2, b2, W3, b3, K[0], K[1], X)
        dW1, db1, dW2, db2, dW3, db3, dK = backward_propogation(Z1, A1, Z2, A2, A3, W3, W2, K[0], K[1], X, Y)
        print(dK)
        W1,b1, W2, b2, W3, b3, K = update_params(W1, b1, W2, b2, W3, b3, K, dW1, db1, dW2, db2, dW3, db3, dK, alpha)
        print("Iteration: ", i)
        print("Accuracy: ", get_accuracy(get_predictions(A3), Y))
    return W1, b1, W2, b2, W3, b3

In [75]:
W1, b1, W2, b2, W3, b3 = gradient_descent(X_train, Y_train, 2, 0.1)

[-6.46659863e-03  5.65731038e+01 -5.29656687e+01]
Iteration:  0
[3 3 3 ... 3 3 3] [2 7 6 ... 5 7 9]
Accuracy:  0.08709756097560975


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


[nan nan nan]
Iteration:  1
[0 3 0 ... 3 3 3] [2 7 6 ... 5 7 9]
Accuracy:  0.06741463414634147
